# Cost of Living Pressures for Low-Income Households
- You may use this template to structure your Capstone Project in Jupyter Notebook.
- Feel free to add or remove sections as needed based on your project scope.
- You are encouraged to include code cells, Markdown explanations, charts, and summaries to clearly demonstrate your analytical thinking and process.

## 1️⃣ Project Title and Introduction:

Give your project a meaningful title. Then briefly describe the context or background of your analysis.

## 2️⃣ Scoping Your Data Analysis Project

- What are the big questions that you are exploring?
- What are the datasets and data columns that you will be exploring?
- What relationships between the data columns will you be exploring?

## 3️⃣Data Preparation

In [294]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

# load datasets
lowincome_cpi = pd.read_csv("data/raw/cpi_2d_lowincome.csv")
statelevel_cpi = pd.read_csv("data/raw/cpi_2d_state_inflation.csv")
population = pd.read_csv("data/raw/population_dun.csv")
MCOICOP = pd.read_csv("data/raw/mcoicop.csv")

# quick inspection function
def inspect_dataset(name, df):
    print(f"\n===== {name} =====")
    print("\nInfo:")
    df.info()
    print("="*50)

# inspect all datasets
inspect_dataset("Low-Income CPI", lowincome_cpi)
inspect_dataset("State-Level CPI", statelevel_cpi)
inspect_dataset("Population by DUN", population)
inspect_dataset("MCOICOP", MCOICOP)




===== Low-Income CPI =====

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      2632 non-null   object 
 1   division  2632 non-null   object 
 2   index     2519 non-null   float64
dtypes: float64(1), object(2)
memory usage: 61.8+ KB

===== State-Level CPI =====

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41888 entries, 0 to 41887
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   state          41888 non-null  object 
 1   date           41888 non-null  object 
 2   division       41888 non-null  object 
 3   inflation_yoy  39424 non-null  float64
 4   inflation_mom  40078 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.6+ MB

===== Population by DUN =====

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 

### Data Cleaning & Data Transformation:

- Handle missing values in both datasets (e.g., missing prices, order dates, or last visit dates).




In [295]:
# Function for check for duplicates row
def identify_duplicates(df):
    duplicates = df.duplicated().sum()
    print("\nDuplicates:", duplicates)

    if duplicates > 0:
         print("Duplicate rows found:")
         display(df[df.duplicated(keep=False)])
         # remove duplicates row
         remove_duplicates(df)

# Function for remove duplicates row
def remove_duplicates(df):
     df.drop_duplicates()

     # verify
     print(f"Number of duplicate rows after removal: {df.duplicated().sum()}")


# check for skewed -0.5 and 0.5 → approximately normal, use mean else median
def check_skewness(col):
    col = pd.to_numeric(col, errors="coerce").dropna()   # drop missing values
    n = len(col)
    if n < 3:  # too few data
        return None

    mean = col.mean()
    std = col.std()
    n = len(col)

    skewness = ((col - mean)**3).sum() / n / (std**3)
    return skewness

# fill the missing data
def impute_data(col,method='mean'):
    if method == 'mean':
        return col.fillna(col.mean())
    elif method == 'median':
        return col.fillna(col.median())

#### low income dataset
# 1. copy
lowincome_cpi_cleaned = lowincome_cpi.copy()

# 1. check for duplicates
identify_duplicates(lowincome_cpi)
# 2. check missing values
print("Missing Values Count:\n", lowincome_cpi.isnull().sum())
# 3. check skeweness
print("Skewness (index):", check_skewness(lowincome_cpi['index']))
# 4. fill
lowincome_cpi_cleaned['index'] = impute_data(lowincome_cpi['index'], method='median')
# 5. verify
print("Missing values after cleaning:\n", lowincome_cpi_cleaned.isnull().sum())



Duplicates: 0
Missing Values Count:
 date          0
division      0
index       113
dtype: int64
Skewness (index): 1.6944989496677305
Missing values after cleaning:
 date        0
division    0
index       0
dtype: int64


In [296]:
#### statelevel_cpi dataset
# 1. copy
statelevel_cpi_cleaned = statelevel_cpi.copy()

# 2. check for duplicates
identify_duplicates(statelevel_cpi)

# 3. remove_duplicates(df)
print("Missing Values Count:\n", statelevel_cpi.isnull().sum())

# 4. check skew
print("Skewness (index):", check_skewness(statelevel_cpi['inflation_yoy']))
print("Skewness (index):", check_skewness(statelevel_cpi['inflation_mom']))

# 5. fill missing data
statelevel_cpi_cleaned['inflation_yoy'] = impute_data(statelevel_cpi['inflation_yoy'], method='median')
statelevel_cpi_cleaned['inflation_mom'] = impute_data(statelevel_cpi['inflation_mom'], method='median')

# 6. verify
print("Missing values after cleaning:\n", statelevel_cpi_cleaned.isnull().sum())


Duplicates: 0
Missing Values Count:
 state               0
date                0
division            0
inflation_yoy    2464
inflation_mom    1810
dtype: int64
Skewness (index): 1.8073331594698339
Skewness (index): 3.5597662693415035
Missing values after cleaning:
 state            0
date             0
division         0
inflation_yoy    0
inflation_mom    0
dtype: int64


In [297]:
# population dataset
# 1. copy
population_cleaned = population.copy()

# 2. check for duplicates
identify_duplicates(population)

# 3. remove_duplicates(df)
print("Missing Values Count:\n", population.isnull().sum())

# 4. check skew
print(population['age'].nunique)
print("Skewness (index):", check_skewness(population['population']))

# 5. fill missing data
population_cleaned['age'] = population['age'].fillna("overall")
population_cleaned['population'] = impute_data(population['population'], method='median')

# 6. verify
print("Missing values after cleaning:\n", population_cleaned.isnull().sum())


Duplicates: 0
Missing Values Count:
 date            0
state           0
parlimen        0
dun             0
sex             0
age           409
ethnicity       0
population    388
dtype: int64
<bound method IndexOpsMixin.nunique of 0       overall
1       overall
2       overall
3       overall
4       overall
         ...   
8995    overall
8996    overall
8997    overall
8998    overall
8999    overall
Name: age, Length: 9000, dtype: object>
Skewness (index): 3.209489941188062
Missing values after cleaning:
 date          0
state         0
parlimen      0
dun           0
sex           0
age           0
ethnicity     0
population    0
dtype: int64


In [298]:
# 1. copy
MCOICOP_cleaned = MCOICOP.copy()

# 2. check for duplicates
identify_duplicates(MCOICOP)

# 3. check missing data
print("Missing Values Count:\n", MCOICOP.isna().sum())
MCOICOP.head()

# No need to fill missing data
# Keep the NaN values because they just show empty levels in the hierarchy, not missing data.
#  just fill na so there is no missing vlaues
MCOICOP_cleaned = MCOICOP_cleaned.fillna('N/A')

# 6. verify
print("Missing values after cleaning:\n", MCOICOP_cleaned.isnull().sum())



Duplicates: 0
Missing Values Count:
 digits        0
division      0
group        14
class        61
subclass    162
desc_en       0
desc_bm       0
dtype: int64
Missing values after cleaning:
 digits      0
division    0
group       0
class       0
subclass    0
desc_en     0
desc_bm     0
dtype: int64


### Data Manipulation and Data Transformation:
- Ensure data types and formatting are consistent.
- Create new columns that are helpful for data analysis



In [299]:
# copy
statelevel_cpi_manipulated = statelevel_cpi_cleaned.copy()
lowincome_cpi_manipulated = lowincome_cpi_cleaned.copy()
population_manipulated = population_cleaned.copy()
MCOICOP_manipulated = MCOICOP_cleaned.copy()

# 1. standardize formating
# a. Align Dates Across Datasets
for df in [statelevel_cpi_manipulated, lowincome_cpi_manipulated, population_manipulated]:
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Verify
print("Date range check:")
# this will check date from earliest date and the latest date
print(statelevel_cpi_manipulated["date"].min(), "to", statelevel_cpi_manipulated["date"].max())
print(lowincome_cpi_manipulated["date"].min(), "to", lowincome_cpi_manipulated["date"].max())
print(population_manipulated["date"].min(), "to", population_manipulated["date"].max())

Date range check:
2010-02-01 00:00:00 to 2025-08-01 00:00:00
2010-01-01 00:00:00 to 2025-08-01 00:00:00
2020-01-01 00:00:00 to 2022-01-01 00:00:00


In [300]:
# ba. Ensure categorical types
statelevel_cpi_manipulated["division"] = statelevel_cpi_manipulated["division"].astype(str)
MCOICOP_manipulated["division"] = MCOICOP_manipulated["division"].astype(str)

# Convert group, class, subclass to string (keep N/A if missing)
for col in ["group", "class", "subclass"]:
    MCOICOP_manipulated[col] = (
        MCOICOP_manipulated[col]
        .fillna("N/A")           # keep missing as "N/A"
        .astype(str)             # convert to string
        .str.replace(r"\.0$", "", regex=True)  # remove .0 from floats like 1.0 → 1
    )

# bb. Ensure numeric types
statelevel_cpi_manipulated["inflation_yoy"] = pd.to_numeric(statelevel_cpi_manipulated["inflation_yoy"], errors="coerce")
statelevel_cpi_manipulated["inflation_mom"] = pd.to_numeric(statelevel_cpi_manipulated["inflation_mom"], errors="coerce")
population_cleaned["population"] = pd.to_numeric(population_manipulated["population"], errors="coerce")

# Verify
print("Data types:")
print(statelevel_cpi_manipulated.dtypes.head())
print(lowincome_cpi_manipulated.dtypes.head())
print(MCOICOP_manipulated.head())

Data types:
state                    object
date             datetime64[ns]
division                 object
inflation_yoy           float64
inflation_mom           float64
dtype: object
date        datetime64[ns]
division            object
index              float64
dtype: object
   digits division group class subclass              desc_en  \
0       1  overall   N/A   N/A      N/A            All items   
1       2       01   N/A   N/A      N/A     Food & Beverages   
2       3       01     1   N/A      N/A  Food away from home   
3       4       01     1    11      N/A  Food away from home   
4       5       01     1    11      111  Food away from home   

                 desc_bm  
0             Semua item  
1      Makanan & Minuman  
2  Makanan di luar rumah  
3  Makanan di luar rumah  
4  Makanan di luar rumah  


In [301]:
# 2. Create calculated columns
# a. create column for population_ratio
population_manipulated["pop_ratio"] = population_manipulated["population"]
population_manipulated["pop_ratio"] = population_manipulated["population"] / population_manipulated["population"].sum()

# Verify
print("Calculated columns preview:")
print(population_manipulated.head())

Calculated columns preview:
        date   state            parlimen               dun     sex      age  \
0 2020-01-01  Perlis  P.001 Padang Besar  N.01 Titi Tinggi    both  overall   
1 2020-01-01  Perlis  P.001 Padang Besar  N.01 Titi Tinggi    both  overall   
2 2020-01-01  Perlis  P.001 Padang Besar  N.01 Titi Tinggi    both  overall   
3 2020-01-01  Perlis  P.001 Padang Besar  N.01 Titi Tinggi  female  overall   
4 2020-01-01  Perlis  P.001 Padang Besar  N.01 Titi Tinggi    male  overall   

    ethnicity  population  pop_ratio  
0     overall      18.696   0.000069  
1     citizen      18.193   0.000067  
2  noncitizen       0.503   0.000002  
3     overall       8.831   0.000033  
4     overall       9.865   0.000037  


In [302]:
# b. Create a mapping for division -> category
division_mapping = {
    "01": "Makanan & Minuman Bukan Alkohol",
    "02": "Minuman Beralkohol & Tembakau",
    "03": "Pakaian & Alas Kaki",
    "04": "Perumahan, Air, Elektrik, Gas & Bahan Api Lain",
    "05": "Perabot, Peralatan Rumah & Penyelenggaraan Rutin Isi Rumah",
    "06": "Kesihatan",
    "07": "Pengangkutan",
    "08": "Komunikasi",
    "09": "Rekreasi & Kebudayaan",
    "10": "Pendidikan",
    "11": "Restoran & Hotel",
    "12": "Perkhidmatan Kewangan & Insurans",
    "13": "Penjagaan Diri, Perlindungan Sosial & Pelbagai Barangan & Perkhidmatan"
}

# Make sure division column is string
MCOICOP_manipulated['division'] = MCOICOP_manipulated['division'].astype(str)

# Create category column
MCOICOP_manipulated['category'] = MCOICOP_manipulated['division'].map(division_mapping)
# For overall items (digits == 1), assign "Semua item"
MCOICOP_manipulated.loc[MCOICOP_manipulated['digits'] == 1, 'category'] = 'Semua item'

# Check the result
print(MCOICOP_manipulated[['digits', 'division', 'category']].head(20))

# performed data quality checks

    digits division                         category
0        1  overall                       Semua item
1        2       01  Makanan & Minuman Bukan Alkohol
2        3       01  Makanan & Minuman Bukan Alkohol
3        4       01  Makanan & Minuman Bukan Alkohol
4        5       01  Makanan & Minuman Bukan Alkohol
5        3       01  Makanan & Minuman Bukan Alkohol
6        4       01  Makanan & Minuman Bukan Alkohol
7        5       01  Makanan & Minuman Bukan Alkohol
8        5       01  Makanan & Minuman Bukan Alkohol
9        5       01  Makanan & Minuman Bukan Alkohol
10       5       01  Makanan & Minuman Bukan Alkohol
11       5       01  Makanan & Minuman Bukan Alkohol
12       4       01  Makanan & Minuman Bukan Alkohol
13       5       01  Makanan & Minuman Bukan Alkohol
14       5       01  Makanan & Minuman Bukan Alkohol
15       5       01  Makanan & Minuman Bukan Alkohol
16       4       01  Makanan & Minuman Bukan Alkohol
17       5       01  Makanan & Minuman Bukan A

In [303]:
# 5. Range checks
statelevel_cpi_manipulated = statelevel_cpi_manipulated[statelevel_cpi_manipulated["inflation_yoy"].between(-20, 20)]
lowincome_cpi_manipulated = lowincome_cpi_manipulated[lowincome_cpi_manipulated["index"].between(50, 200)]

# Verify
print("Inflation range after filtering:")
print("State-level:", statelevel_cpi_manipulated["inflation_yoy"].min(), statelevel_cpi_manipulated["inflation_yoy"].max())
print("Low-income:", lowincome_cpi_manipulated["index"].min(), lowincome_cpi_manipulated["index"].max())

Inflation range after filtering:
State-level: -20.0 20.0
Low-income: 85.3 176.9


 low_income_cpi_full['index_3m_avg'] = low_income_cpi_full.groupby('division')['index'].transform(lambda x: x.rolling(3).mean())
### Data Joining:

- Join the datasets using a unique identifier.
- Perform groupby to uncover relationships between variables.


In [304]:
# Join 1: State CPI + Population
# merge
merged_cpi_population = statelevel_cpi_manipulated.merge(
    population_manipulated,
    # population_manipulated[["state", "date", "pop_ratio"]],
    on=["state", "date"],
    how="left"
)

# Drop Null column
merged_cpi_population2 = merged_cpi_population.dropna(subset=["pop_ratio"])

# iff needed to fill
# print("Missing Values Count:\n", merged_cpi_population2.isnull().sum())
# print("Skewness (index):", check_skewness(merged_cpi_population2['pop_ratio']))
# lowincome_cpi_cleaned['index'] = impute_data(merged_cpi_population2['pop_ratio'], method='median')

# Verify
print(merged_cpi_population2.head())



     state       date division  inflation_yoy  inflation_mom       parlimen  \
119  Johor 2020-01-01  overall            1.8            0.2  P.140 Segamat   
120  Johor 2020-01-01  overall            1.8            0.2  P.140 Segamat   
121  Johor 2020-01-01  overall            1.8            0.2  P.140 Segamat   
122  Johor 2020-01-01  overall            1.8            0.2  P.140 Segamat   
123  Johor 2020-01-01  overall            1.8            0.2  P.140 Segamat   

                  dun     sex      age   ethnicity  population  pop_ratio  
119  N.01 Buloh Kasap    both  overall     overall     31.9560   0.000118  
120  N.01 Buloh Kasap    both  overall     citizen     20.6115   0.000076  
121  N.01 Buloh Kasap    both  overall  noncitizen      3.2210   0.000012  
122  N.01 Buloh Kasap  female  overall     overall     14.2120   0.000053  
123  N.01 Buloh Kasap    male  overall     overall     17.7440   0.000066  


In [305]:
# Join 2: Low-Income CPI + MCOICOP (for category mapping)
merged_lowincome = pd.merge(
    lowincome_cpi_manipulated, MCOICOP_manipulated,
    # lowincome_cpi_manipulated, MCOICOP_manipulated[["division", "category"]],
    on="division", how="left"
)

# Verify
# Filter rows where division == "02"
filtered = merged_lowincome[merged_lowincome["division"] == "02"]

# Show first few rows
print(filtered.head())



            date division  index  digits group class subclass  \
14476 2010-01-01       02   98.3       2   N/A   N/A      N/A   
14477 2010-01-01       02   98.3       3    21   N/A      N/A   
14478 2010-01-01       02   98.3       4    21   211      N/A   
14479 2010-01-01       02   98.3       5    21   211     2110   
14480 2010-01-01       02   98.3       4    21   212      N/A   

                             desc_en                     desc_bm  \
14476  Alcoholic Beverages & Tobacco  Minuman Alkohol & Tembakau   
14477            Alcoholic beverages       Minuman keras/alkohol   
14478              Spirits & liquors        Arak & minuman keras   
14479              Spirits & liquors        Arak & minuman keras   
14480                           Wine                        Wain   

                            category  
14476  Minuman Beralkohol & Tembakau  
14477  Minuman Beralkohol & Tembakau  
14478  Minuman Beralkohol & Tembakau  
14479  Minuman Beralkohol & Tembakau  
14480

In [306]:
# Join 3: State CPI + MCOICOP (Rubric: Data integration with lookup table)
merged_state = pd.merge(
    statelevel_cpi_manipulated, MCOICOP_manipulated,
    # statelevel_cpi_manipulated, MCOICOP_manipulated[["division", "category"]],
    on="division", how="left"
)

# Verify
print("Merged state CPI + MCOICOP:")
print(merged_state.head())

Merged state CPI + MCOICOP:
   state       date division  inflation_yoy  inflation_mom  digits group  \
0  Johor 2010-02-01  overall            1.2            0.0       1   N/A   
1  Johor 2010-03-01  overall            1.2            0.0       1   N/A   
2  Johor 2010-04-01  overall            1.2            0.0       1   N/A   
3  Johor 2010-05-01  overall            1.2            0.2       1   N/A   
4  Johor 2010-06-01  overall            1.2            0.2       1   N/A   

  class subclass    desc_en     desc_bm    category  
0   N/A      N/A  All items  Semua item  Semua item  
1   N/A      N/A  All items  Semua item  Semua item  
2   N/A      N/A  All items  Semua item  Semua item  
3   N/A      N/A  All items  Semua item  Semua item  
4   N/A      N/A  All items  Semua item  Semua item  


## 4️⃣ Exploratory Data Analysis

1. Summarize datasets to understand overall structure and basic stats.
2. Focus only on low-income households and relevant periods (2010–2023).
3. Segment analysis by state, category, and demographics.



+### Descriptive Analysis:

- Explore overall descriptive analysis.
- Filter subsets to answer big questions.

### Data Visualisation:
- Visualise data in graphs to better understand the data.



```
# This is formatted as code
```

## 5️⃣ Data Insights

- Summarize your main takeaways. What patterns or trends did you find?